In [29]:
pip install tensorflow


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [52]:
import numpy as np
import pandas as pd
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.dicts.emoticons import emoticons
from nltk.stem import WordNetLemmatizer
import nltk
from mittens import GloVe
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cosine
from sklearn.model_selection import train_test_split
from keras.models import Sequential

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
from keras.optimizers import RMSprop,Adam


In [2]:
df=pd.read_csv('Lemetized_Tokenized_HateSpeechDataset.csv')
df

/var/folders/y8/p7vw8wn924qchdnzwptmkrp40000gn/T/ipykernel_2116/1924614298.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('Lemetized_Tokenized_HateSpeechDataset.csv')


,Unnamed: 0,Content,Label,Content_int,Tokens
0,0,denial of normal the con be asked to comment o...,1,"[146715, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,...","['denial', 'of', 'normal', 'the', 'con', 'be',..."
1,1,just by being able to tweet this insufferable ...,1,"[146715, 14, 15, 16, 17, 7, 18, 19, 20, 21, 22...","['just', 'by', 'being', 'able', 'to', 'tweet',..."
2,2,that is retarded you too cute to be single tha...,1,"[146715, 28, 29, 30, 26, 31, 32, 7, 5, 33, 28,...","['that', 'is', 'retarded', 'you', 'too', 'cute..."
3,3,thought of a real badass mongol style declarat...,1,"[146715, 35, 1, 24, 36, 37, 38, 39, 40, 1, 41,...","['thought', 'of', 'a', 'real', 'badass', 'mong..."
4,4,afro american basho,1,"[146715, 46, 47, 48, 146714]","['afro', 'american', 'basho']"
...,...,...,...,...,...
440901,440901,crash another movie from left field i have to ...,0,"[146715, 1780, 649, 1673, 347, 701, 1702, 80, ...","['crash', 'another', 'movie', 'from', 'left', ..."
440902,440902,i why do not you debate first before starting ...,0,"[146715, 80, 186, 340, 76, 26, 6407, 566, 2608...","['i', 'why', 'do', 'not', 'you', 'debate', 'fi..."
440903,440903,removal of i reverted the removal of the above...,0,"[146715, 18731, 1, 80, 48182, 3, 18731, 1, 3, ...","['removal', 'of', 'i', 'reverted', 'the', 'rem..."
440904,440904,i have unblocked you eddie as i discussed on u...,0,"[146715, 80, 87, 20515, 26, 10684, 273, 80, 95...","['i', 'have', 'unblocked', 'you', 'eddie', 'as..."


In [3]:
df = df[:10000]
len(df)

10000

### Vectorization

In [4]:
documents = [content for content in df['Content']]
# Step 1: Build the co-occurrence matrix
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1, 1))
X = vectorizer.fit_transform(documents)
cooccurrence = (X.T @ X).toarray()
# Set the diagonal to zero to ignore self-co-occurrences
np.fill_diagonal(cooccurrence, 0)
# Step 2: Initialize and train the GloVe model
glove_model = GloVe(n=100, max_iter=100, learning_rate=0.05)
glove_embeddings = glove_model.fit(cooccurrence)
# Get word-to-index dictionary from CountVectorizer
vocab = vectorizer.get_feature_names_out()
word_to_index = {word: idx for idx, word in enumerate(vocab)}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Iteration 100: error 2283.0031

In [5]:
# Example: Get vector for a specific word
word = 'cat'
if word in word_to_index:
    vector = glove_embeddings[word_to_index[word]]
    print(f"Vector for '{word}':\n", vector)

def most_similar(word, embeddings, word_to_index, top_n=5):
    if word not in word_to_index:
        return []
    word_vector = embeddings[word_to_index[word]]
    similarities = [
        (other_word, 1 - cosine(word_vector, embeddings[word_to_index[other_word]]))
        for other_word in word_to_index.keys()
        if other_word != word
    ]
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]


similar_words = most_similar(word, glove_embeddings, word_to_index)
print(f"Most similar words to '{word}':\n", similar_words)

Vector for 'cat':
 [-0.34816657 -0.03738416 -0.25963539  0.14351346  0.34098776 -0.1075993
  0.29054128  0.04179224 -0.11461408 -0.27021417  0.42767185 -0.0410276
  0.04105763  0.03632908 -0.3198685  -0.31135141 -0.59486438  0.21701385
 -0.02903507 -0.06898059  0.25498591  0.18755094 -0.0771486   0.06295336
 -0.29198261 -0.39825811 -0.59918913 -0.28103442  0.10881959 -0.45792378
 -0.21875949  0.14523527  0.72499449  0.13871453  0.74333978  0.24149316
  0.08245373  0.08885383  0.11699921  0.2462823   0.31963666  0.00273491
 -0.04595449  0.1665612  -0.24526992  0.50742002  0.14687805  0.43850036
  0.21083558 -0.1462869   0.45812963  0.43015371  0.10887179 -0.09926219
  0.07630221 -0.03923153  0.15954663 -0.65973721  0.13582076 -0.17349741
 -0.11625287  0.52869544  0.15782611 -0.21276808  0.20357589  0.11992559
  0.02919122  0.1408833   0.04625325  0.19637261  0.02228543  0.23546023
  0.33873199 -0.05982465 -0.03412218 -0.30969766 -0.57096957 -0.25716229
 -0.35321666  0.06542494  0.208876

In [181]:
glove_embeddings=np.load('glove_embeddings[:10000].npy')

### Sequences

In [182]:
X= df['Tokens']
Y = df['Label']

In [183]:
X

0                                                                                                                                                                              ['denial', 'of', 'normal', 'the', 'con', 'be', 'asked', 'to', 'comment', 'on', 'tragedies', 'an', 'emotional', 'retard']
1                                                                                                                                                                   ['just', 'by', 'being', 'able', 'to', 'tweet', 'this', 'insufferable', 'bullshit', 'proves', 'trump', 'a', 'nazi', 'you', 'vagina']
2                                                                                                                                                                                                          ['that', 'is', 'retarded', 'you', 'too', 'cute', 'to', 'be', 'single', 'that', 'is', 'life']
3                                                                                                               

In [184]:
sequences = [[word_to_index[word] for word in tokens if word in word_to_index] for tokens in documents]

In [185]:
sequences

[[2802,
  3486,
  7554,
  5618,
  1,
  6413,
  7874,
  3973,
  7554,
  7874,
  9090,
  6857,
  1,
  6413,
  11338,
  5041,
  3486,
  1620,
  7874,
  7554,
  767,
  3486,
  1,
  9813,
  6232,
  3486,
  2802,
  11338,
  7874,
  1620,
  7874,
  6857,
  6857,
  3486,
  7554,
  11338,
  7874,
  7554,
  11338,
  9090,
  1,
  4585,
  3486,
  2802,
  5618,
  3486,
  9813,
  1,
  7554,
  3486,
  6857,
  7874,
  11338,
  5618,
  7874,
  7554,
  1,
  6413,
  9090,
  3486,
  11338,
  1,
  9090,
  2802],
 [6059,
  12071,
  9813,
  11338,
  767,
  12888,
  767,
  3486,
  5618,
  7554,
  4585,
  1,
  767,
  6413,
  3486,
  11338,
  7874,
  11338,
  12457,
  3486,
  3486,
  11338,
  11338,
  5041,
  5618,
  9813,
  5618,
  7554,
  9813,
  12071,
  3973,
  3973,
  3486,
  9090,
  1,
  767,
  6413,
  3486,
  767,
  12071,
  6413,
  6413,
  9813,
  5041,
  5618,
  11338,
  8144,
  9090,
  7874,
  12287,
  3486,
  9813,
  11338,
  9090,
  12071,
  6857,
  8144,
  1,
  7554,
  1,
  5618,
  12888,
  7874,
 

In [186]:
vocab_size = len(vocab) + 1
max_seq_length = max([len(seq) for seq in sequences])
X_pad = pad_sequences(sequences, maxlen=max_seq_length, padding = 'post', truncating = 'post')
X_pad

array([[ 2802,  3486,  7554, ...,     0,     0,     0],
       [ 6059, 12071,  9813, ...,     0,     0,     0],
       [11338,  5041,     1, ...,     0,     0,     0],
       ...,
       [11338,  5041,     1, ...,     0,     0,     0],
       [12888,  7874, 12071, ...,     0,     0,     0],
       [ 6413,  3486, 11338, ...,     0,     0,     0]], dtype=int32)

In [187]:
vocab.shape

(12991,)

### Test Training Split

In [188]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, Y, test_size=0.30, random_state=42)

In [134]:
X_train

array([[ 2802,  7874, 11338, ...,     0,     0,     0],
       [ 5618,     1,  6857, ...,     0,     0,     0],
       [11338,  5041,     1, ...,     0,     0,     0],
       ...,
       [ 5618,  3973,  9813, ...,     0,     0,     0],
       [ 5618,   767,     1, ...,     0,     0,     0],
       [ 9813,  7874,  1620, ...,     0,     0,     0]], dtype=int32)

In [189]:
y_train

9069    0
2603    0
7738    0
1579    0
5058    0
       ..
5734    0
5191    0
5390    0
860     0
7270    0
Name: Label, Length: 7000, dtype: object

##  Training LSTM Model

In [193]:
# Build the LSTM model

embedding_dim = glove_embeddings.shape[1]

model = Sequential()

model.add(Embedding(input_dim=vocab_size - 1, 
                            output_dim=embedding_dim, 
                            weights=[glove_embeddings], 
                            trainable=False))

model.add(LSTM(units=50, return_sequences=True))
    
model.add(Dropout(0.2))
    
model.add(LSTM(units=50,return_sequences=False))
    
model.add(Dropout(0.2))
    
model.add(Dense(units=1, activation='sigmoid'))
    
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [194]:
X_train = np.array(X_train).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
X_test = np.array(X_test).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)

In [ ]:
batch_size = 64

model.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/10
 48/110 ━━━━━━━━━━━━━━━━━━━━ 2:21 2s/step - accuracy: 0.8426 - loss: 0.4665

## Testing LSTM Model

In [175]:
loss, acc = model.evaluate(X_test, y_test)

print(acc)

94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.8451 - loss: 0.1312
0.8519999980926514


In [177]:
sequence = [word_to_index[word] for word in 'fuck you lebron james'.split(' ') if word in word_to_index]

In [178]:
testing = pad_sequences([sequence], maxlen=max_seq_length, padding = 'post', truncating = 'post')

In [179]:
testing

array([[ 4543, 12936,  6547,  6079,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [180]:
model.predict(testing)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step


array([[0.13611126]], dtype=float32)